In [4]:
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from fastai.vision.all import *
from PIL import Image

from torch import optim
import torch.nn as nn
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import AdamW
import cv2

import gc
import glob
import inspect
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial

from skimage.metrics import structural_similarity as ssim
from skimage import color, transform
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')
set_seed(3, reproducible=True)

In [5]:
path = '/kaggle/input/zindidata/'

In [6]:
trn_path = f'{path}images/images/'

In [7]:
train            = pd.read_csv(f'{path}Train.csv')
test             = pd.read_csv(f'{path}Test.csv')
SampleSubmission = pd.read_csv(f'{path}SampleSubmission.csv')

In [9]:
Train = train.copy()
Test = test.copy()
FOLD = 4
skf = StratifiedKFold(FOLD, shuffle=True, random_state=3)
X = Train.drop(columns='damage')
y = Train.damage

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)

In [10]:
damage = Train.damage.unique()

In [11]:
class_weights = torch.FloatTensor([1.3, 1.2, 1.4, 1.0, 2.8]).cuda()

In [12]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.45, 0.46, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()

    def before_call(self, b, split_idx):
        self.idx = split_idx

    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']

        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.InvertImg(p=0.3),
        A.VerticalFlip(p=.4),
        A.HorizontalFlip(p=.5),
        A.RandomRotate90(p=.2),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.3,
            rotate_limit=45, border_mode=0, p=.6
        ),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.RandomBrightnessContrast(brightness_limit=(-0.15,0.2), contrast_limit=(-0.1, 0.1), p=0.5),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.3),
            max_w_size=int(IMG_WIDTH*0.3),
            num_holes=4,
            p=.7,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = []
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [Resize(224, method='squish'), AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [17]:
def get_dls_teacher(mean_std='imagenet'):
    clas_block = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = trn_path),
                       get_y = ColReader('damage'),
                       splitter = RandomSplitter(valid_pct = 0.25, seed = 3),
                       item_tfms = get_item_tfms(mean_std), 
                       batch_tfms = aug_transforms(min_scale= 0.75), 
                      )
    dls = clas_block.dataloaders(Train, bs=64)
    dls.rng.seed(3)

    return dls

In [14]:
def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = trn_path),
                       get_y = ColReader('damage'),
                       splitter = MaskSplitter(Train.fold == fold),
                       item_tfms = get_item_tfms(mean_std), #Resize(320, method = 'squish'),
                       batch_tfms = aug_transforms(min_scale= 0.75), #aug_transforms(size = 128, min_scale = 0.75)
                      )
    dls = clas_block.dataloaders(Train, bs=72)
    dls.rng.seed(3)

    return dls

In [15]:
def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

In [18]:
data = get_dls_teacher('imagenet')
teacher = vision_learner(data, 'vit_base_patch16_224', loss_func = CrossEntropyLossFlat(), metrics=accuracy).to_fp16()
teacher.freeze()
teacher.fit_one_cycle(9, 0.01)

epoch,train_loss,valid_loss,accuracy,time
0,1.081555,0.953423,0.591990,09:53
1,0.962499,0.915083,0.629738,08:19
2,0.829199,0.755842,0.699708,07:53


KeyboardInterrupt: 

In [19]:
teacher.freeze_to(2)
teacher.fit_one_cycle(5, 0.01)

epoch,train_loss,valid_loss,accuracy,time
0,0.779586,0.715588,0.712598,09:35
1,0.771138,0.713436,0.714439,09:06


KeyboardInterrupt: 

In [20]:
teacher.freeze()
teacher.save("teacher")
free_memory([data])

In [21]:
predictions = 0
for fold in range(FOLD):
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'fold-{fold}'
    dls = get_dls(fold, 'imagenet')
    teacher = load_learner("teacher")
    learn = vision_learner(dls, 'convnext_nano.in12k',
                           loss_func = CrossEntropyLossFlat(),
                           metrics=[accuracy,Precision(average='macro'), Recall(average='macro'), RocAuc(average='macro')]).to_fp16()
    distillation_callback = KnowledgeDistillation(learn, teacher)
    learn.fit_one_cycle(7, 1e-3, callbacks=[distillation_callback])
    interp = ClassificationInterpretation.from_learner(learn)
    losses,idxs = interp.top_losses()
    len(dls.valid_ds)==len(losses)==len(idxs)
    interp.plot_confusion_matrix(figsize=(7,7))

    test_dl = learn.dls.test_dl(trn_path + Test['filename'])
    preds1 , _ = learn.tta(dl=test_dl)
    preds1 = F.softmax(preds1, dim=1)
    submission_sf = pd.DataFrame({

    'ID': Test['ID'],
    'DR': preds1[:, 0].squeeze().numpy(),
    'G' : preds1[:, 1].squeeze().numpy(),
    'ND': preds1[:, 2].squeeze().numpy(),
    'WD': preds1[:, 3].squeeze().numpy(),
    'other': preds1[:, 4].squeeze().numpy(),
    })
    if fold==0:
        predictions = submission_sf
    else:
        merged_df = pd.merge(predictions, submission_sf, on='ID', suffixes=('_df1', '_df2'))

        # Extract columns that you want to average
        columns_to_average = ['DR_df1', 'G_df1', 'ND_df1', 'WD_df1', 'other_df1', 'DR_df2', 'G_df2',  'ND_df2', 'WD_df2', 'other_df2']

        # Take the average for each entry
        merged_df['DR'] = merged_df[columns_to_average].filter(like='DR').mean(axis=1)
        merged_df['G'] = merged_df[columns_to_average].filter(like='G').mean(axis=1)
        merged_df['ND'] = merged_df[columns_to_average].filter(like='ND').mean(axis=1)
        merged_df['WD'] = merged_df[columns_to_average].filter(like='WD').mean(axis=1)
        merged_df['other'] = merged_df[columns_to_average].filter(like='other').mean(axis=1)

        # Resulting DataFrame with averages
        predictions = merged_df[['ID', 'DR', 'G', 'ND', 'WD', 'other']]
        print(predictions)

    free_memory([teacher, learn, preds1, dls])

*************************Fold 0*************************


FileNotFoundError: [Errno 2] No such file or directory: 'teacher'

In [ ]:
predictions.to_csv('convnext_nano_vit_base_kd_fold4_.csv', index=False)

In [ ]:
predictions